In [126]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce
from pyspark.sql.functions import col
from pyspark.sql import SQLContext
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import category_encoders as ce

In [127]:
def preProcessData(dataset):
    
    # only keeping successful and failed projects
    projects = dataset[(dataset['state'] == 'failed') | (dataset['state'] == 'successful')]
    
    # deleting unnecesary columns
    projects.drop('ID',axis=1,inplace=True)
    projects.drop('name',axis=1,inplace=True)
    projects.drop('category',axis=1,inplace=True)
    projects.drop('currency',axis=1,inplace=True)
    projects.drop('goal',axis=1,inplace=True)
    projects.drop('pledged',axis=1,inplace=True)
    projects.drop('usd pledged',axis=1,inplace=True)
    
    # Finding length of the project 
    projects['launched'] = pd.to_datetime(projects['launched']).dt.to_period("M")
    projects['deadline'] = pd.to_datetime(projects['deadline']).dt.to_period("M")
    
    # Creating a new columns with Campaign total months
    projects['total_months'] = projects['deadline']- projects['launched']
    projects['total_months'] = projects['total_months'].astype(int)
    projects.drop('launched',axis=1,inplace=True)
    projects.drop('deadline',axis=1,inplace=True)
    
    # Moving the outcome variable to last and encoding it
    state = projects.pop('state')
    projects['state']=state
    projects['state']= LabelEncoder().fit_transform(projects['state'])
    
    # One Hot Encoding the categorical features
    onehotencoder = ce.OneHotEncoder(cols = ['main_category','country'])
    projects = onehotencoder.fit_transform(projects)
    
    # Avoiding dummy variable trap
    projects.drop('main_category_-1',axis=1,inplace=True)
    projects.drop('country_-1',axis=1,inplace=True)

  
    labels = projects.columns

    X = projects.iloc[:10000, :-1].values
    y = projects.iloc[:10000, -1].values

    # Splitting the dataset into the Training set and Test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

    # Feature Scaling
    sc_X = StandardScaler()
    X_train[:,38:] = sc_X.fit_transform(X_train[:,38:])
    X_test[:,38:] = sc_X.transform(X_test[:,38:])

    training_data = pd.concat([pd.DataFrame(X_train),pd.Series(y_train)],axis=1)
    test_data = pd.concat([pd.DataFrame(X_test),pd.Series(y_test)],axis=1)

    training_data.columns = labels
    test_data.columns= labels
    
    return training_data,test_data

In [128]:
dataset = pd.read_csv('ks-projects-201801.csv')
training_data, test_data = preProcessData(dataset)

C:\Users\Tejaswini\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Tejaswini\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Tejaswini\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Tejaswini\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 

In [129]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [130]:
#compute sigmoid(x,W)
def compute_h(x,W):
    x = np.append([1],[x])
    #print(x_vector)
    #perform dot product of x and W
    z=np.dot(x,W)
    return sigmoid(z)

In [131]:
#compute 
def compute_gradient(point):
    #create feature vector x
    x_vector = np.append([1],[point[1:]])
    #print(x_vector.shape)
    #create response vector y
    w_vector= point[0]
    #print(y_vector.shape)
    #perform dot product of x and y
    gradient = w_vector * x_vector
    return gradient

In [132]:
#for point in devZipX_Rdd.collect():
    #create feature vector x
 #   x_vector = np.append([1],[point[1:]])
  #  print(x_vector.shape)
    #create response vector y
   # y_vector= point[0]
    #print(y_vector.shape)
    #perform dot product of x and y
    #gradient = y_vector * x_vector
    #print(gradient)

In [133]:
#def gradient(X, y, predicted):
#    print(predicted.shape[0])
#    dw = np.dot(X.T , (predicted - y))
#    return dw

In [134]:
def compute_pred(X,W):
    z = np.dot(X, W)
    return sigmoid(z)[:,0]

In [152]:
#ones = np.ones(len(training_data))
train_X = training_data.iloc[:,:-1].as_matrix()
train_y = training_data.iloc[:,-1].as_matrix()
W = np.zeros([train_X.shape[1]+1, 1])
#print(train_X.shape)
learning_rate = 0.5
#print(type(W))

In [153]:
#train_XRdd = sqlContext.createDataFrame(train_X)
train_XRdd = sc.parallelize(train_X)
#train_XRdd.collect()
train_yRdd = sc.parallelize(train_y)

In [137]:
#compute h(x,W)
#pred_yRdd =train_XRdd.map(lambda x : compute_h(x,W))
# combine predicted y value rdd and actual y value rdd
#predTrain_Rdd = pred_yRdd.zip(train_yRdd) 
# compute difference between predicted y and actual y value
#devRdd = predTrain_Rdd.map(lambda x : (x[0] - x[1]))
#len(devRdd.collect())
#train_XRdd.take(2)

In [154]:
for i in range(20):
    #compute h(x,W)
    pred_yRdd =train_XRdd.map(lambda x : compute_h(x,W))
    # combine predicted y value rdd and actual y value rdd
    predTrain_Rdd = pred_yRdd.zip(train_yRdd) 
    # compute difference between predicted y and actual y value
    devRdd = predTrain_Rdd.map(lambda x : (x[0] - x[1]))
    # compute gradient : (predicted - y)X
    devX_Rdd = devRdd.zip(train_XRdd)
    gradRdd = devX_Rdd.map(lambda lst : ('keyA',compute_gradient(lst)))
    dwRdd = gradRdd.reduceByKey(lambda x,y :  np.add(x,y))
    gradientRdd = dwRdd.mapValues(lambda dw : learning_rate * dw)
    #update weights
    weightRdd = sc.parallelize(W)
    #devRdd.lookup('keyA')[0].shape
    dRdd = sc.parallelize(gradientRdd.lookup('keyA')[0])
    newRdd = weightRdd.zip(dRdd) 
    wRdd = newRdd.map(lambda x : (x[0] - x[1]))
    W = np.stack(wRdd.collect())

In [139]:
#devX_Rdd = devRdd.zip(train_XRdd)
#len(devZipX_Rdd.collect())
# compute gradient : (predicted - y)X
#gradRdd = devX_Rdd.map(lambda lst : ('keyA',compute_gradient(lst)))
#dwRdd = gradRdd.reduceByKey(lambda x,y :  np.add(x,y))
#gradientRdd.collect()


In [140]:
#gradientRdd = dwRdd.mapValues(lambda dw : learning_rate * dw)
#print(len(gradientRdd.collect()))

In [141]:
#update weights
#weightRdd = sc.parallelize(W)
#devRdd.lookup('keyA')[0].shape
#dRdd = sc.parallelize(gradientRdd.lookup('keyA')[0])
#newRdd = weightRdd.zip(dRdd) 
#Rdd = newRdd.map(lambda x : (x[0] - x[1]))
#type(np.stack(Rdd.collect()))

In [142]:
#for i in range(10):
 #   pred_Rdd =train_XRdd.map(lambda x : compute_z(x,W))
  #  devRdd = pred_Rdd.zip(train_yRdd) 
   # devRdd = devRdd.map(lambda x : (x[0] - x[1]))
    #devZipX_Rdd = devRdd.zip(train_XRdd)
    #gradRdd = devZipX_Rdd.map(lambda lst : ('keyA',compute_gradient(lst)))
    #gradientRdd = gradRdd.reduceByKey(lambda x,y :  np.add(x,y))
    #devRdd = gradientRdd.mapValues(lambda dw : learning_rate * dw)
    #weightRdd = sc.parallelize(W)
    #devRdd.lookup('keyA')[0].shape
    #dRdd = sc.parallelize(devRdd.lookup('keyA')[0])
    #newRdd = weightRdd.zip(dRdd) 
    #newRdd = newRdd.map(lambda x : (x[0] - x[1]))
    #W = np.stack(newRdd.collect())

In [160]:
print(W)

[[ -1.40765740e+03]
 [ -3.41301607e+02]
 [ -1.70757715e+02]
 [  2.08069351e+02]
 [ -3.04995139e+02]
 [ -2.41508316e+02]
 [ -1.29715350e+02]
 [ -1.46902387e+02]
 [  2.43532464e+02]
 [ -3.53817971e+02]
 [  3.75475811e+02]
 [  1.34805164e+01]
 [ -7.18716548e+01]
 [ -4.93317589e+02]
 [  1.52425748e+02]
 [ -1.46453560e+02]
 [  1.29718205e+01]
 [ -8.38421524e+02]
 [ -9.81356992e+01]
 [ -1.01056438e+02]
 [  9.00029550e+00]
 [ -1.12910711e+02]
 [ -5.22236627e+01]
 [ -2.09494257e+01]
 [ -5.60247604e+01]
 [  2.61633218e+00]
 [ -8.09291324e+00]
 [ -1.24621655e+01]
 [ -1.17391557e+01]
 [  8.97522452e+00]
 [ -2.42650908e+01]
 [ -1.47979654e+01]
 [ -2.52612922e+01]
 [ -5.74999825e+00]
 [ -7.45018467e+01]
 [ -1.75344707e+00]
 [  6.12502377e+00]
 [  1.10000000e+01]
 [  0.00000000e+00]
 [  1.97066505e+03]
 [  1.41418479e+03]
 [ -1.02120215e+03]
 [ -1.25185335e+02]]


In [144]:
#for x in train_XRdd.collect():
  
 #   x = np.append([1],[x])
    #print(x_vector)
  #  z=np.dot(x,W)
   # print(sigmoid(z))

In [155]:
test_Xm = np.column_stack((np.ones(len(test_data)), test_data.iloc[:,:-1]))
test_y = test_data.iloc[:,-1]     

In [156]:
test_X = test_data.iloc[:,:-1].as_matrix()
test_y = test_data.iloc[:,-1].as_matrix()

In [157]:
test_XRdd = sc.parallelize(test_X)
test_yRdd = sc.parallelize(test_y)

In [161]:
pred_Rdd =test_XRdd.map(lambda x : compute_h(x,W))
pred_y = pred_Rdd.collect()
#pred_y = compute_pred(test_Xm,W)

In [163]:
print("Accuracy: ", 100 - np.mean(np.abs(pred_y - test_y)) * 100, "%")

Accuracy:  59.95677531 %
